# Iterated Hessian Sketching

## Problem Setting: Linear Regression

* Starting with the classical linear regression situation, we make the following data and modeling assumptions:

$$\begin{aligned}
&\text{Model: } Y = X\beta + \varepsilon\\
&X: n\times p \text{ design matrix;}\\
&\beta: p \times 1 \text{ coefficient vector;}\\
&\varepsilon \sim N(0, I_n \otimes \sigma^2), \,\, n \times 1 \text{ random error vector.}\\
\end{aligned}$$

The goal of the regression is to find the estimated coefficient vector to minimize the 
squared-error loss function, equivalent to maximizing the likelihood in this case.  Denoting the optimal solution as $\hat \beta$, the target solution can be written

$$\hat \beta = \arg \min_{\beta} \|Y - X\beta\|_2^2.$$

In this simple setting, there is a closed-form solution:

$$\hat \beta = (X'X)^{-1}X'Y.$$

While this is a convenient theoretical expression and practical for most situations, problems may arise in evaluating this expression directly due to the computational cost and possible numerical instability of computing $(X'X)^{-1}$.  Thus, it may be better to fit the model via a matrix decomposition algorithm, or with an optimization method such as gradient descent or Newton-Raphson.  Moreover, in the case of generalized linear models (GLMs), no closed-form solution for $\hat \beta$ exists, and it is necessary to apply an optimization method or some other alternative.

## The Newton-Raphson Method

Considering the Newton-Raphson method in general, suppose we wish to minimize the loss function $Q(\theta)$ over the space $\Theta$ of valid $p$-dimensional parameter vectors $\theta$.  We assume for now that the problem is unconstrained, so that $\Theta = \mathbb{R}^p$.

This is a second-order method, requiring computation of the gradient $\nabla Q(\theta)$ and the Hessian $\nabla^2 Q(\theta)$.  The standard NR update formula is

$$\theta_{n + 1} = \theta_n - \gamma [\nabla^2 Q(\theta)]^{-1}\nabla Q(\theta),$$

where $\gamma \in (0, 1)$ is a step-size parameter that can be adjusted to help ensure convergence.  This formula is derived from applying a second-order Taylor approximation to the cost function.  It offers an advantage over alternatives like gradient descent by incorporating second-order information, with the tradeoff of increased computational complexity in calculating and inverting the Hessian.

Newton-Raphson is the workhorse of statistical model fitting, and is often employed for coefficient estimation due to its simplicity in implementation, its typically fast convergence, and in its flexibility across many different problem settings and model designs.  However, inverting the Hessian can be very expensive and quickly becomes prohibitive in large data settings.  Naive matrix inversion with the Gauss-Jordan elimination method require $O(n^3)$ time for an $n \times n$ matrix.  More recently developed methods have asymptotic complexity as low as $O(n^{2.373})$, but carry very large constant factors, making their application impractical.  In any case, it is clear that the Hessian inversion is the computational bottleneck in the Newton-Raphson algorithm, and that any reduction in the size of the Hessian matrix will yield significant time savings.  This is the motivation for the Iterated Hessian Sketching (IHS) method.

## Classical Sketching

Before examining the more refined IHS procedure, we consider the "classical" sketching method, which is not a useful method in itself, but serves as a simple conceptual introduction.

The basic idea of sketching procedures in the regression setting is the application of randomized projections to the data in order to reduce the computational burden required for finding the optimal solution.  Specifically, instead of optimizing the original cost function

$$\hat \beta = \arg \min_{\beta} \|Y - X\beta\|_2^2,$$

we solve the sketched version, 

$$\hat \beta = \arg \min_{\beta} \|S(Y - X\beta)\|_2^2,$$

where $S$ is a random "sketching" matrix with dimension $m \times n$, with sketching dimension $m < n$.  This approach can be seen as a form of subsampling in a rough sense, although sketching is more general, producing random linear combinations of observations.  The statement of the "method" here is simple, but there are many crucial underlying questions for this approach to be practical.  Chiefly,


1. Does this sketching method or some variation yield a model fitting algorithm that is both more computationally efficient and sufficiently accurate in approximating the full regression solution?
2. What sketching dimension $m$ is required to guarantee a reasonable approximate fit?
3. Are there classes of random projection matrices that produce the best results?

Unfortunately, it turns out that classical sketching does not easily prodouce real computational gains due to the required tradeoff in accuracy.  Accuracy can be improved by iterating this approach, but the computation-accuracy tradeoff is insufficient to ensure practical improvements in.


## Hessian Sketching



## Classes of Random Projection Matrices

### Sub-Gaussian Sketches

The simplest choice for random sketching matrix $S \in \mathbb{R}^{m \times n}$ is to draw from a matrix normal $N_{m\times n}(0, I_n, I_m)$.  This can be generalized to include sub-Gaussian sketches.  A random vector $s \in \mathbb{R}^n$ is defined to be 1-sub-Gaussian if

$$P[\langle s, u\rangle \geq \varepsilon\|u\|_2] \leq e^{-\varepsilon^2/2} \hspace{0.5cm} \forall\,\varepsilon \geq 0.$$

Sub-Gaussian sketches are theoretically simple and a natural choice, but carry no inherent structure, and therefore require $O(mnp)$ time to compute the product $SX$.

### Randomized Orthonormal Systems

To achieve computational gains beyond what is offered by sub-Gaussian sketches, we instead consider using sketches from randomized orthonormal systems (ROS).  Specifically, we consider two orthonormal basis systems here, those defined by Hadamard matrices; and those defined by Fourier bases.  In both cases, there exist fast transformation algorithms that can reduce the computation time $O(n p\log m)$ when computing $SX$.

#### Hadamard Transform

The orthonormal Hadamard matrix $H_n$ of size $2^n \times 2^n$ can be defined recursively as

$$H_1 = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1\\1 & -1\end{pmatrix}\\$$
$$H_n = \frac{1}{\sqrt{2}} \begin{pmatrix} H_{n - 1} & H_{n - 1}\\ H_{n - 1} & -H_{n - 1}\end{pmatrix}.$$

The rows (or columns) of these matrices define an orthonormal basis for $\mathbb{R}^n$.  The Hadamard transform defined by these matrices is actually an example of a generalized Discrete Fourier Transform (DFT).

#### Fourier Transform

Naturally then, we can also consider the classical DFT itself as forming an orthonormal basis to randomly select from.  The DFT matrix is given by

$$W_n = \frac{1}{n}
\begin{pmatrix}
1 & 1 & 1 & \cdots & 1\\
1 & \omega & \omega^2 & \cdots & \omega^{n - 1}\\
1 & \omega^2 & \omega^4 & \cdots & \omega^{2(n - 1)}\\
\vdots & \vdots & \vdots & \vdots & \vdots\\
1 & \omega^{n - 1} & \omega^{2(n - 1)} & \cdots & \omega^{(n - 1)(n - 1)}
\end{pmatrix},
$$

where $\omega = e^{_2\pi i/n}$ is the primitive $n$th root of unity.

#### ROS Sampling Scheme

Having chosen an orthonormal basis system, a sketching matrix $S\in \mathbb{R}^{m \times n}$ is constructed by 
sampling i.i.d. rows of the form

$$s' = \sqrt{n}e_j'HD \hspace{0.5cm} \text{with probability 1/n for $j = 1, \cdots, n$,}$$

where $e_j$ a Euclidean basis vector chosen uniformly at random and $D = diag(\nu)$ is a diagonal matrix of randomly chosen Rademacher variables $\nu \in \{-1, 1\}^n$.



## GLMs

In the GLM setting, we have observations $Y_i \sim (\mu_i, V(\mu_i)), i = 1, \cdots, n$ from an exponential family distribution, where $V(\mu_i)$ is the variance for observation $i$, which depends on the mean level for that observation in general.

![](http://www.maa.org/sites/default/files/images/cms_upload/Color001000192.JPG)